## Section 1: Operator Overview 
* Using the most recent month

In [ ]:
import _report_utils
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [ ]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
name = "SBMTD Schedule"
organization_name = "Santa Barbara Metropolitan Transit District"
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [ ]:
type(selected_date)

### Route typology

In [ ]:
display(
    Markdown(
        f"""{organization_name} ran <b>{op_profiles_df.operator_n_routes.values[0]}</b> 
        unique routes in {selected_date.month}/{selected_date.year}."""
    )
)

In [ ]:
def route_typology(df: pd.DataFrame):
    route_type_cols = [
        "operator_n_routes",
        "n_downtown_local_routes",
        "n_rapid_routes",
        "n_local_routes",
    ]
    df = df[route_type_cols]
    df2 = df.T.reset_index()
    df2 = df2.rename(columns={"index": "route_type", 0: "total_routes"})
    df3 = df2.loc[df2.route_type != "operator_n_routes"]
    return df3

In [ ]:
route_typology_data = route_typology(op_profiles_df)

In [ ]:
basic_pie_chart(
    route_typology_data, "route_type", "total_routes", "Total Routes by Typology"
)

### Operator Stops

In [ ]:
display(
    Markdown(
        f"""This operator served <b>{op_profiles_df.operator_n_stops.values[0]}</b> different stops and each stop received   <b>{op_profiles_df.operator_arrivals_per_stop.values[0]}</b> visits, totaling to <b>{op_profiles_df.operator_n_arrivals.values[0]}</b> arrivals."""
    )
)

### Operator Area Coverage

In [ ]:
op_routes_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_routes_map}.parquet"

In [ ]:
op_routes_gdf = gpd.read_parquet(
    op_routes_url,
    filters=[[("name", "==", name), ("service_date", "==", selected_date)]],
)

In [ ]:
op_routes_gdf = gpd.read_parquet(
    op_routes_url)

In [ ]:
display(
    Markdown(
        f"""{organization_name} provided <b>{op_profiles_df.operator_route_length_miles.values[0]}</b> miles of public transit.
        The average length of a route is <b>{op_routes_gdf.route_length_miles.mean().round(2)}</b> miles.
        """
    )
)

In [ ]:
route_length_df = (
    op_routes_gdf[["route_combined_name", "route_length_miles"]]
    .sort_values(by=["route_length_miles"])
    .iloc[[0, -1]]
)

In [ ]:
basic_bar_chart(route_length_df, "route_length_miles", "route_combined_name", "Longest and Shortest Route")

In [ ]:
def get_counties():
    ca_gdf = "https://opendata.arcgis.com/datasets/8713ced9b78a4abb97dc130a691a8695_0.geojson"
    my_gdf = to_snakecase(gpd.read_file(f"{ca_gdf}"))[["county_name", "geometry"]]

    return my_gdf

In [ ]:
def counties_served(gdf:gpd.GeoDataFrame)->pd.DataFrame:
    ca_counties = get_counties()
    counties_served = gpd.sjoin(
    gdf,
    ca_counties.to_crs(gdf.crs),
    how="inner",
    predicate="intersects",
).drop(columns="index_right")
    
    counties_served = counties_served[["county_name"]].drop_duplicates()
    display(counties_served.style.hide().hide(axis="columns"))

In [ ]:
display(
    Markdown(
        f"""{organization_name} provides public transit in the following counties.
        """
    )
)

In [ ]:
counties_served(op_routes_gdf)

In [ ]:
display_all_routes(op_routes_gdf)

### Total Service Hours

In [ ]:
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [ ]:
monthly_scheduled_service_df_agg = _report_utils.summarize_monthly(name)

In [ ]:
_report_utils.single_bar_chart_dropdown(
    monthly_scheduled_service_df_agg,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Total Service Hours per Month",
    "full_date",
)